# Particle Swarm Optimisation for Graph Neural Network Architecture Search

In [231]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# from particle import Particle
# from swarm import Swarm

This GNN can handle different types of convolutional layers, and both node and graph classification.
The build_conv_model method determines which type of convolutional layer to use for the given task, a graph convolutional network for node classificationtion (GCNConv) and a graph isomorphism network for graph classification (GINConv).
This model is made of 3 covolution layers followed by mean pooling in the case of graph classification, followed by 2 fully connected layers.
Sing our goal here is classification, we use a negative log-likelihood loss function.

List of the changeable hyperparameters:

    - hidden_dim (dimmension of hidden layers)

    - number of convs layers

    - number of lns layers (normalisation layers)
    

In [232]:
class GNNStack(nn.Module):
    def __init__(self, input_dim, output_dim, task='node', hidden_num = 2, hidden_dim = 32):
        super(GNNStack, self).__init__()
        self.task = task
        self.convs = nn.ModuleList() #convolution operations
        self.lns = nn.ModuleList() #normalisation operations
        self.convs.append(self.build_conv_model(input_dim, hidden_dim))
        # adding convolution an normalisation layers
        for l in range(hidden_num):
            self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))
            self.lns.append(nn.LayerNorm(hidden_dim))

        # self.lns.append(nn.LayerNorm(hidden_dim))


        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25), 
            nn.Linear(hidden_dim, output_dim)) # adding 2 linear layers 
        if not (self.task == 'node' or self.task == 'graph'):
            raise RuntimeError('Unknown task.')

        self.dropout = 0.25
        self.num_layers = 3

    def build_conv_model(self, input_dim, hidden_dim):
        # refer to pytorch geometric nn module for different implementation of GNNs.
        # create different types of GCNConv according to the problem to solve
        if self.task == 'node':
            return pyg_nn.GCNConv(input_dim, hidden_dim)
        else:
            return pyg_nn.GINConv(nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                  nn.ReLU(), nn.Linear(hidden_dim, hidden_dim)))

    def forward(self, data):
        # x = feature matrix = number of nodes * number of node feature dimensions,
        # edge_index = list of the edges in the graph,
        # batch = batch of a graph
        x, edge_index, batch = data.x, data.edge_index, data.batch

        if data.num_node_features == 0: #if there is no feature, use a constant feature
          x = torch.ones(data.num_nodes, 1)

        for i in range(self.num_layers): # ,create num_layers convolution layers
            x = self.convs[i](x, edge_index)
            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            if not i == self.num_layers - 1:
                x = self.lns[i](x)

        if self.task == 'graph': # if it is a graph classification task, do a pooling
            x = pyg_nn.global_mean_pool(x, batch)

        x = self.post_mp(x)

        return emb, F.log_softmax(x, dim=1)

    def loss(self, pred, label):
        return F.nll_loss(pred, label)

pyg_nn.GCNConv and pyg_nn.GINConv are instances of MessagePassing, They define a single layer of graph convolution, which can be decomposed into:
- Message computation
- Aggregation
- Update
- Pooling

With CustomConv, we have an exemple of how to subclass the pytorch geometric MessagePassing class to derive a new model  rather than using existing GCNConv and GINConv.
We use the MessagePassing's key building blocks:
- aggr='add': The aggregation method to use ("add", "mean" or "max").
- propagate(): The initial call to start propagating messages. Takes in the edge indices and any other data to pass along (e.g. to update node embeddings).
- message(): Constructs messages to node i. Takes any argument which was initially passed to propagate().
- update(): Updates node embeddings. Takes in the output of aggregation as first argument and any argument which was initially passed to propagate().

In [233]:
class CustomConv(pyg_nn.MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(CustomConv, self).__init__(aggr='add')  # "Add" aggr egation.
        self.lin = nn.Linear(in_channels, out_channels)
        self.lin_self = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Add self-loops to the adjacency matrix.
        edge_index, _ = pyg_utils.remove_self_loops(edge_index)

        # Transform node feature matrix.
        self_x = self.lin_self(x)
        #x = self.lin(x)

        # start the propagation of the message
        return self_x + self.propagate(edge_index, size=(x.size(0), x.size(0)), x=self.lin(x))

    def message(self, x_i, x_j, edge_index, size):
        # Compute messages
        # x_j has shape [E, out_channels]

        row, col = edge_index
        deg = pyg_utils.degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]
        return aggr_out

Training the model with forward propagation and back propagation. 
For node classification, we split nodes into training and testing sets.
Same thing for graph classification, we use 80% of the graphs for training and the remainder for testing.

In [234]:
def train(dataset, task, writer, hidden_num , hidden_dim):
    trigger = 0
    prev_acc = 0.0

    if task == 'graph':
        data_size = len(dataset)
        # split data  into traning and testing set
        loader = DataLoader(dataset[:int(data_size * 0.8)], batch_size=64, shuffle=True)
        test_loader = DataLoader(dataset[int(data_size * 0.8):], batch_size=64, shuffle=True)
    else:
        test_loader = DataLoader(dataset, batch_size=64, shuffle=True)
        loader = test_loader

    # build model
    model = GNNStack(max(dataset.num_node_features, 1), dataset.num_classes, task=task, hidden_num=hidden_num, hidden_dim=hidden_dim)
    opt = optim.Adam(model.parameters(), lr=0.01)
    
    # train for 200 epochs
    print("hidden_num = " + str(hidden_num))
    print("hidden_dim = " + str(hidden_dim))
    for epoch in range(200):
        total_loss = 0
        model.train()
        for batch in loader:
            # print(batch.train_mask, '----')
            opt.zero_grad()
            # get the prediction and the excpeted label
            embedding, pred = model(batch)
            # pred = model(batch)
            label = batch.y
            if task == 'node':
                pred = pred[batch.train_mask]
                label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        writer.add_scalar("loss", total_loss, epoch)

        
        test_acc = test(test_loader, model)
        # print("Epoch {}. Loss: {:.4f}. Test accuracy: {:.4f}".format(
        #     epoch, total_loss, test_acc))
        # writer.add_scalar("test accuracy", test_acc, epoch)

        if test_acc <= prev_acc:
            trigger += 1

        if trigger >= 10:
            print("the model is not learning anymore.")
            print("Epoch {}. Loss: {:.4f}. Test accuracy: {:.4f}".format(
            epoch, total_loss, test_acc))
            writer.add_scalar("test accuracy", test_acc, epoch)
            break
        prev_acc = test_acc

    return model, loss.item()

For the CiteSeer/Cora node classification task, there is only one graph, so we use masking to determine validation and test set. In graph classification, a subset of graphs is considered as a validation/test graph

In [235]:
def test(loader, model, is_validation=False):
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad(): # avoid gradient computing for faster results
            emb, pred = model(data)
            pred = pred.argmax(dim=1)
            label = data.y

        if model.task == 'node':
            mask = data.val_mask if is_validation else data.test_mask
            # node classification: only evaluate on nodes in test set
            pred = pred[mask]
            label = data.y[mask]
            
        correct += pred.eq(label).sum().item()
     
    if model.task == 'graph':
        total = len(loader.dataset) 
    else:
        total = 0
        for data in loader.dataset:
            total += torch.sum(data.test_mask).item()
    return correct / total

In [236]:
# get_ipython().system_raw(
#     'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#     .format("./log")
# )
# get_ipython().system_raw('./ngrok http 6006 &')
# !curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [237]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset = dataset.shuffle()
task = 'graph'

model = train(dataset, task, writer, 2, 32)

hidden_num = 2
hidden_dim = 32
the model is not learning anymore.
Epoch 22. Loss: 1.6679. Test accuracy: 0.2083


In [238]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

dataset = Planetoid(root='/tmp/cora', name='cora')
task = 'node'

model = train(dataset, task, writer, 2, 32)

hidden_num = 2
hidden_dim = 32
the model is not learning anymore.
Epoch 24. Loss: 0.2112. Test accuracy: 0.7350


In [239]:
# color_list = ["red", "orange", "green", "blue", "purple", "brown", "black"]

# loader = DataLoader(dataset, batch_size=64, shuffle=True)
# embs = []
# colors = []
# for batch in loader:
#     emb, pred = model(batch)
#     embs.append(emb)
#     colors += [color_list[y] for y in batch.y]
# embs = torch.cat(embs, dim=0)

# xs, ys = zip(*TSNE().fit_transform(embs.detach().numpy()))
# plt.scatter(xs, ys, color=colors)

///////////////////
///////////////////
///////////////////
///////////////////

In [249]:
import copy
import random

class Particle:
    def __init__(self):
        self.hidden_dim = 2
        self.hidden_num = 2
        # self.parameters = NN_parameters()
        self.cognitiveCoef = 1 # can be changed
        self.socialCoef = 1 # can be changed
        self.informantList = list()
        self.informants_best_err = -1
        self.best_err = -1
        self.best_wb = []
        self.informants_best = [2, 2]
        self.err = -1  # Current error (set to -1 at start
        self.velocity_hidden_num = random.random()
        self.velocity_hidden_dim = random.random()

    def setInformants(self, swarm, informantNum, index):
        banned_index = []
        i = 0
        swarm_buffer = copy.deepcopy(swarm)
        banned_index.append(index)
        while i < informantNum:
            informant_chosen = np.random.randint(0, len(swarm_buffer))
            if informant_chosen in banned_index:
                continue
            self.informantList.append(swarm[informant_chosen])
            banned_index.append(informant_chosen)
            i += 1

    def set_informant_best(self):
        for informer in self.informantList:
            if informer.best_err < self.informants_best_err or self.informants_best_err == -1:
                self.informants_best_err = informer.best_err
                self.informants_best = informer.best_wb
    
    def check_error(self, loss):
        self.err = 0
        if self.err < self.best_err or self.best_err == -1:
            self.best_err = self.err
            self.best_wb = [self.hidden_num, self.hidden_dim]

    def update_velocity(self):
        inertia_weight = 1

        r1 = random.random()
        r2 = random.random()

        vel_cog = self.cognitiveCoef * r1 * (self.best_wb[0] - self.hidden_num)
        vel_soc = self.socialCoef * r2 * (self.informants_best[0] - self.hidden_num)
        self.velocity_hidden_num = inertia_weight * self.velocity_hidden_num + vel_soc + vel_cog

        #Change the velocity values for B1
        r1 = random.random()
        r2 = random.random()

        vel_cog = self.cognitiveCoef * r1 * (self.best_wb[1] - self.hidden_dim)
        vel_soc = self.socialCoef * r2 * (self.informants_best[1] - self.hidden_dim)
        self.velocity_hidden_dim = inertia_weight * self.velocity_hidden_dim + vel_soc + vel_cog

    # Update the hidden num and dim
    def change_wb(self):
        self.hidden_num = int(self.velocity_hidden_num + self.hidden_num)
        self.hidden_dim = int(self.velocity_hidden_dim + self.hidden_dim)

In [250]:
class Swarm:
    def __init__(self, informants_number, particle_number):

        self.informants_number = informants_number
        self.swarm = list()
        self.particle_number = particle_number

        for i in range(self.particle_number):
            new_particle = Particle()
            self.swarm.append(new_particle)
        
        for j in range(len(self.swarm)):
            self.swarm[j].setInformants(self.swarm, self.informants_number, j)
        

    def Optimise(self):
        #Run Optimisation
        for p in range(0,self.particle_number):
            # Find best informants
            self.swarm[p].set_informant_best()
            # Update velocities
            self.swarm[p].update_velocity()
            # Apply velocities to weights and biases
            self.swarm[p].change_wb()

    # For every particle, creates a neural network from the particles weights and biases. And than calculate the output values of the neural network
    def forward_prop(self, dataset, task):
        for p in range(0, int(self.particle_number)):
            print("Particle num = " + str(p))
            # informants_best[0] = hidden_num
            # informants_best[1] = hidden_dim
            
            # writer = SummaryWriter("./log/" + "|hidden_num = "
            #                         + str(self.swarm[p].informants_best[0])
            #                         +"|hidden_dim = "
            #                         + str(self.swarm[p].informants_best[1]) +"|")
            writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

            # print(self.swarm[p].hidden_num)
            # print(self.swarm[p].hidden_dim)
            model, loss = train(dataset, task, writer, int(self.swarm[p].hidden_num), (self.swarm[p].hidden_dim))
            self.swarm[p].check_error(loss)

    # Return the best error of the entire swarn
    def get_best(self):
        swarm_best_err = -1
        for p in self.swarm:
            if p.best_err < swarm_best_err or swarm_best_err == -1:
                swarm_best_err = p.best_err

        return swarm_best_err

    def plot(self, y):
        plt.plot(y)
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.ylim([0,100])
        plt.show()

In [251]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset = dataset.shuffle()
task = 'graph'

# 3 informants, 6 particles
swarm = Swarm(3, 6)
for i in range(10):
    print("Epoch = " + str(i))
    swarm.forward_prop(dataset, task)
    swarm.Optimise()
    best_err = swarm.get_best()

Epoch = 0
Particle num = 0
hidden_num = 2
hidden_dim = 2


/home/tristan/anaconda3/envs/thesis/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


the model is not learning anymore.
Epoch 10. Loss: 1.7971. Test accuracy: 0.1083
Particle num = 1
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 12. Loss: 1.7987. Test accuracy: 0.1083
Particle num = 2
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 11. Loss: 1.7940. Test accuracy: 0.1083
Particle num = 3
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 12. Loss: 1.7912. Test accuracy: 0.1083
Particle num = 4
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 10. Loss: 1.7940. Test accuracy: 0.1083
Particle num = 5
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 10. Loss: 1.7920. Test accuracy: 0.1083
Epoch = 1
Particle num = 0
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 12. Loss: 1.7868. Test accuracy: 0.1083
Particle num = 1
hidden_num = 2
hidden_dim = 2
the model is not learning anymore.
Epoch 11. Loss: 1.7939. Test accuracy: 0.1083
Particle num 

KeyboardInterrupt: 